The first block of codes below allows you to install The OMDB package into the program. OMDB stands for Open Movie Database, it is a "RESTful movie service used to obtain movie information", REST stands for  representational state transfer. I also install the pyautogui package which allows me to use GUI to generate blocks and have less human error.

In [1]:
import sys##installing PyPi libraries
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install omdb
#installing OMDB
!{sys.executable} -m pip install pyautogui
#installing pyautogui for message boxes

The block of code below is used to access the OMDB through the unique API key, which you get once you register to the website, I made this extra function so it is easier to read the other functions used which are not from standard built in packages.

In [2]:
import omdb

def APIkey(key):##installing the API key
    omdb.set_default('apikey', key)

The split function is used to extract the contents of the textfile "History.txt" and put it into a list, split by the parameter which is sent from the location it is called from. For example "title" can be sent to split the textfile by titles of movies, keeping all of the movies information together in one block in the list

In [3]:
def split(info): ##Spliting the history textfile by the paramter given and returning the list
    file_opener = open("History.txt", "r")
    unsorted_history = file_opener.read()
    file_opener.close()
    information = unsorted_history.split(info)
    return information

When the search function is called, the title of the movie that the user is searching for is passed across from the main method to this function. It then makes the movie title lowercase to prevent duplicate errors from user entered mistakes such as searching "Mad Max" and then searching "mad max". The split function is then called to put the contents of our textfile (which holds all the movies information of which you have searched beforehand) into a list, split by the paramter sent across, which in this case is "title". This will put the entirety of each movie's information into each posititon of the list "information". It then checks if the movietitle the user searched is already in the textfile, if it is it will display that part of the list, which contains all the information on the movie and will turn the boolean into false, which will stop the program from checking the OMDB for the contents of the movie. If the movie title cannot be found in the textfile, boolean "check" will remain True, and the program will check the OMDB for the movie. If the movie doesn't exist, it will return an empty dictionary, and so I use the len() function to check that the movie is on the OMDB, if it exists, it will append to the textfile, using json to dump the contents of the movie information into the textfile, and print the contents of the movie, otherwise it will explain that the movie you have searched for does not exist. 

In [4]:
import json
import pyautogui

def search(movietitle):##searching for a movie title
    movietitle = movietitle.lower()
    count = 0
    check = True##checking to see if in text file before checking internet
    information = split("title")
    for word in information:
        if movietitle in word.lower():
            count += 1
            if count <= 1:
                pyautogui.alert(text="{\"title"+word[:-2], title="Movie Database LTD.", button="continue")
                check = False
    
    if check:##If it is not in the textfile
        APIkey('62c6d5ed')
        name = omdb.get(title=movietitle)##searching movie name
        if len(name) > 0:##If the movie does exist
            history_write(name)
            pyautogui.alert(text=name, title="Movie Database LTD.", button="continue")
        else:##If the movie does not exist
            pyautogui.alert(text=movietitle + " does not exist!", title="Movie Database LTD.", button="continue")

the clear function is used to clear the contents of the text file. When this function is called, it opens the textfile sent across as a write function, which will clear the contents of the textfile, it will then close the textfile, leaving an empty textfile. This function is used for clearing your search history for new users or at the users request.

In [5]:
def clear(textfile):##clear history of textfile
    open(textfile, "w").close()

history_write is a function that appends to the "History" textfile with the contents sent across by the other function. 

In [6]:
def history_write(name):
    with open('History.txt', 'a') as history_file:##adding movie details to text file, movie details passed as a parameter
        history_file.write(json.dumps(name)+",\n")
        history_file.close()

password_write is a function that appends to the "Password" textfile to allow the last user to continue if needed

In [7]:
def password_write(password):
    with open('Password.txt', 'a') as history_file:##adding password to text file, password passed as a parameter
        history_file.write(json.dumps(password))
        history_file.close()

specific_search is a function that works similar to the search function, but allows the user to search for a specific piece of information on the movie, so that they do not need to look through large amounts of information for one specific piece of information. This function retrieves the movie name and the information they want to retireve from this movie, it does not need to check if th information is an option as they are given to the user as button choices and so they cannot put in custom searches for that specific piece of information. It will find the movie and the information, it will check if the movie is real and if so will check if the movie has already been found in the textfile, if it is, it will print that specific information on the movie as well as the title, if the user requests the title, then it will just display the title once, if the movie isn't in the textfile, it will call the history_write function to append to the textfile. If the user searches for the poster, it will use the webbrowser functions to open the poster for the user on the internet. If the movie doesn't exist, it will explain to the user that the movie doesn't exist.

In [8]:
import webbrowser##for opening the internet from the program

def specific_search(movietitle, information):##searching a specific criteria of movie
    movietitle = movietitle.lower()
    check = True
    APIkey('62c6d5ed')
    name = omdb.get(title=movietitle)##searching movie name
    if len(name) > 1:##checking movie exists
        if information == "title":
            pyautogui.alert(text="Title: " + name["title"], title="Movie Database LTD.", button="continue")
            information = split(",")
            for word in information:##checking movie in textfile
                if movietitle in word.lower():
                    check = False
            if check:##If it isn't in textfile
                history_write(name)
        else:##If it isn't title they looking for
            if information == "poster":
                webbrowser.open(str(name[information]))##opens up the poster image on the internet using the link provided by OMDB
                information = split(",")
                for word in information:##checking for movie in textfile
                    if movietitle in word.lower():
                        check = False
                if check:##If movie isn't in textfile
                    history_write(name)
            else:##If the movie isn't a textfile or a poster
                pyautogui.alert(text="Title: " + movietitle + "\n" +information + ": " + str(name[information]), title="Movie Database LTD.", button="continue")
                information = split(",")
                for word in information:##checking for movie in textfile
                    if movietitle in word.lower():
                        check = False
                if check:##If the movie isn't in textfile
                    history_write(name)
    else:##If the movie does not exist on the OMDB
        pyautogui.alert(text=movietitle + " does not exist!", title="Movie Database LTD.", button="continue")

the history function will display the titles of the movies that the user has searched for so far, this is done by retrieving the information on the textfile and spliting it in a list by the "," which is done by calling the split function. It will then go through the list and look for "title", if it finds the title, it will print that title and move on. If there is no search history, which is checked by the len function on the list, it will explain that nothing has been searched yet.

In [9]:
def history():##returning the titles of the history in the textfile
    
    check = False
    history_store = ""
    information = split(",")
    if len(information) > 1:##checking their is a search history
        for word in information:
            if "title" in word:
                word = word.replace("{","")##removing the { to deal with typing error
                history_store = history_store + "\n" + word
                check = True
    else:##If you haven't searched anything
        pyautogui.alert(text="You haven't searched anything yet!", title="Movie Database LTD.", button="continue")
    
    if check:##posting the movie titles of there is a history
        pyautogui.alert(text= history_store, title="Movie Database LTD.", button="continue")

This is the main method, that is why it isn't a function. I put it at the bottom so that all the other methods can be run in the beginning before running the main method to avoid running into errors of the system saying that a certain function does not exist. This main method starts by welcoming the user to the Movie Database and asking if they are a new user. The user must choose the button Yes or No. If they are new the program will run the clear function to remove the previous users search history and will ask the user to make a password which can be used later to remove their search history. If the user chooses no to being a new user, it will simply move on.<br><br>
Once the user has selected 'Yes' or 'No', it will then ask the user whether they would like to:<br> 
A. Search  for a movie<br> 
B. Clear their search history<br>
C. Show their search history<br>
D. Find specific information on a movie<br>
E. Exit the program.<br><br>
The user will select the letter for the option they want. If they select:<br>
A. The user will be asked to enter the name of the movie they would like and it will be sent across to the search function<br>
B. The clear function will be called to remove the users history, once they provide the correct password<br>
C. The history function will be called to display the titles of the previous movies searched<br>
D. The user will be asked for the name and information on the movie they want and that information will be sent to specific_search<br>
E. The user will end the program and a message will be displayed to thank the user for using the program.

In [17]:
import pyautogui##allows for message boxes ##main method

new_user = pyautogui.confirm("Welcome to Movie Database LTD. Are you a new user?", title = "Movie Database LTD.", buttons = ["Yes","No"])
if new_user == "No":
    file_opener = open("Password.txt", "r")
    unsorted_history = file_opener.read()
    file_opener.close()
    if len(unsorted_history) < 1 or unsorted_history == "Default":
        password = "Default"
        clear("Password.txt")
if new_user == "Yes":
    clear("History.txt")##gets rid of previous users search history
    clear("Password.txt")##gets rid of previous users password
    while True:##password encrypts history so people can't see it
        password = pyautogui.password(text="Enter a password to store history (minimum length = 6)", title="Movie Database LTD.", default='', mask="*")
        if password == None:
            pyautogui.alert(text="Thank you for using Movie Database LTD.", title="Movie Database LTD.", button="Exit")
            sys.exit("You pressed cancel")
        else:
            if password == '':##if the user doesn't put a password
                pyautogui.alert(text="No password entered, please try again!", title="Movie Database LTD.", button="Try Again.")
            else:
                if len(password) < 6:##if the password isn't long enough
                    pyautogui.alert(text="Password too short, please try again!", title="Movie Database LTD.", button="Try Again.")
                else:##if password is good
                    password_write(password)
                    break
##if they are not a new user it will simply continue from the last time including password

while True:
    options = pyautogui.confirm("Please select an option.\nA: Find information on a movie\nB: Clear search history\nC: Show search history\nD: Search specific information about a movie.\nE: Exit Program.", title = "Movie Database LTD.",buttons = ["A","B","C","D","E"])
    if options == "A":
        movietitle = pyautogui.prompt("Enter the title of the movie you want to find", title = "Movie Database LTD.")
        if movietitle == None:
            pyautogui.alert("You pressed cancel.", title = "Movie Database LTD.", button = "continue")
        else:
            search(movietitle.strip())
        
    else:
        if options == "B":
            password_check = pyautogui.password(text="Enter your password to clear history", title="Movie Database LTD.", default='', mask="*")
            if password == password_check:
                clear("History.txt")
            else:
                pyautogui.alert(text="Incorrect password entered!", title="Movie Database LTD.", button="OK")
        else:
            if options == "C":
                password_check = pyautogui.password(text="Enter your password to view history", title="Movie Database LTD.", default='', mask="*")
                if password == password_check:
                    history()
                else:
                    pyautogui.alert(text="Incorrect password entered!", title="Movie Database LTD.", button="OK")
                    
            else:
                if options == "D":
                    movietitle = pyautogui.prompt(text="Enter the title of the movie you want to find",title="Movie Database LTD.")
                    if movietitle == None:
                        pyautogui.alert(text="You pressed cancel", title="Movie Database LTD.", button="continue")
                    else:
                        information = pyautogui.confirm("Enter the information you want to retrieve from "+ movietitle, title="Movie Database LTD.", buttons = ['title','year','rated','released','runtime','genre','director','writer','actors','plot','language','awards','poster','ratings','metascore','imdb_rating','imdb_votes','imdb_id','type','dvd','box_office','production','website','response'])
                        specific_search(movietitle.strip(),information)
                else:
                    if options == "E":
                        break

pyautogui.alert(text="Thank you for using Movie Database LTD!", title="Movie Database LTD.", button="Exit")

'Exit'

In order to test the program, I will be using normal, abnormal and extreme data to see how my program responds to each data type.<br><br>
As the program begins to run, the user is asked if he is a new user or not.<br>
Normal data: "Yes"<br>
Abnormal data: N/A<br>
Extreme data: N/A<br><br>
When run, these are the outputs:<br>
Normal data: Ran perfectly and cleared history. Asked user to provide a password:<br>
Password inputs:<br>
Normal data: "Byron123"<br>
Abnormal data: "Byron1"<br>
Extreme data: "!^@!W", "mon"<br><br>
Password outputs:<br>
Normal data: Runs perfectly, moves onto next part of program<br>
Abnormal data: Runs perfectly, moves onto next part of program<br>
Extreme data: Says "Password too short, please try again!", "No password entered, please try again!"<br><br>
Abnormal data: N/A<br>
Extreme data: N/A<br><br>
Once the user has stated whether he is new or not, the user will be asked what they would like to do. They must select "A" - "E".<br>
Normal data: "A","B","C","D","E"<br>
Abnormal data: N/A<br>
Extreme data: N/A<br><br>
The outputs are as follows:<br>
Normal data: A: The program promts the user to "Enter the title of the movie you want to find". For this option the user will test with three types of data:<br><br>
Normal data:"Avatar"<br>
Abnormal data:"tHE mATRiX"<br>
Extreme data:"Not a real movie does not exist!"<br><br>
Outputs:<br>
Normal data:"{'title':'Avatar', 'year': '2009', 'rated': 'PG-13', 'released': '18 Dec 2009', 'runtime': '162 min', 'genre': 'Action, Adventure, Fantasy', 'director': 'James Cameron', 'writer': 'James Cameron', 'actors': 'Sam Worthington, Zoe Saldana, Sigourney Weaver', 'plot': 'A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.', 'language': 'English, Spanish', 'country': 'United States', 'awards': 'Won 3 Oscars. 89 wins & 131 nominations total', 'poster': 'https://m.media-amazon.com/images/M/MV5BZDA0OGQxNTItMDZkMC00N2UyLTg3MzMtYTJmNjg3Nzk5MzRiXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_SX300.jpg', 'ratings': [{'source': 'Internet Movie Database', 'value': '7.8/10'}, {'source': 'Rotten Tomatoes', 'value': '82%'}, {'source': 'Metacritic', 'value': '83/100'}], 'metascore': '83', 'imdb_rating': '7.8', 'imdb_votes': '1,189,326', 'imdb_id': 'tt0499549', 'type': 'movie', 'dvd': '22 Apr 2010', 'box_office': '\\$760,507,625', 'production': 'N/A', 'website': 'N/A', 'response': 'True'}"<br>
Abnormal data:"{'title': 'The Matrix', 'year': '1999', 'rated': 'R', 'released': '31 Mar 1999', 'runtime': '136 min', 'genre': 'Action, Sci-Fi', 'director': 'Lana Wachowski, Lilly Wachowski', 'writer': 'Lilly Wachowski, Lana Wachowski', 'actors': 'Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss', 'plot': 'When a beautiful stranger leads computer hacker Neo to a forbidding underworld, he discovers the shocking truth--the life he knows is the elaborate deception of an evil cyber-intelligence.', 'language': 'English', 'country': 'United States, Australia', 'awards': 'Won 4 Oscars. 42 wins & 51 nominations total', 'poster': 'https://m.media-amazon.com/images/M/MV5BNzQzOTk3OTAtNDQ0Zi00ZTVkLWI0MTEtMDllZjNkYzNjNTc4L2ltYWdlXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_SX300.jpg', 'ratings': [{'source': 'Internet Movie Database', 'value': '8.7/10'}, {'source': 'Rotten Tomatoes', 'value': '88\%'}, {'source': 'Metacritic', 'value': '73/100'}], 'metascore': '73', 'imdb_rating': '8.7', 'imdb_votes': '1,851,767', 'imdb_id': 'tt0133093', 'type': 'movie', 'dvd': '15 May 2007', 'box_office': '$172,076,928', 'production': 'N/A', 'website': 'N/A', 'response': 'True'}"<br>
Extreme data:"not a real movie"<br><br>
B: Runs a password check to see if the passwords are the same:<br><br>
Normal data: "Byron123"<br>
Abnormal data: "bYRON 123"<br>
Extreme data: ""<br><br>
Outputs:<br><br>
Normal data: Runs perfectly and clears history<br>
Abnormal data: "Incorrect password entered!" as it is case sensitive, doesn't clear history<br>
Extreme data: "Incorrect password entered!", doesn't clear history<br><br>
C: shows your history. For example if you have searched "Avatar", "Avatar", "Ben 10"<br>
Output:<br>
title : Avatar<br>
title : Ben 10<br><br>
D: It asks you to enter the name of the movie you are searching and you choose a button for the information you want on the movie:<br>
For example input:<br><br>
Normal data:"Avatar", "plot"<br>
Abnormal data:"tHE mATRiX", "runtime"<br>
Extreme data:"Not a real movie", "actors"<br><br>
The output is as follows:<br>
Normal data: Title: avatar<br>
plot: A paraplegic Marine dispatches to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home<br>
Abnormal data: Title: the matrix<br>
runtime: 136 min<br>
Extreme data: Not a real movie does not exist!<br><br>
E: The program will exit and a message will be displayed saying thank you for using the program.<br><br><br>
Errors I encountered and dealt with while creating this program:<br><br>
When I first began writing this program, I had to do research on what an OMDB is, how it's data is returned and the most effective way to deal with it. I managed to figure out that the data is returned as a dictionary when using the .get() method. Using this method allowed me to find specific information easily, but made putting data into the textfile difficult as it would only put the first part of the dictionary. To counter this problem I imported json and used json.dump() which allowed me to put everything into the textfile. When I tried to retrieve the work from the textfile I thought I would be able to use the same method as normal in terms of using a dictionary search, but it came back as a string. To get around this issue I decided to split the string up by the comma and then search that list for the information I wanted. The next issue I found was that when I looked up a movie, if one didn't exist it would simply return an empty dictionary, which would then result in errors when searching that dictionary for information, I managed to fix this issue by using len() to see if the dictionary is empty or not. Finally I found that many unneccessary errors were caused from incorrect user input, after doing extensive research and trying many different ways of countering the users errors, I decided the best way to deal with it was to limit the user interaction. I did this by importing pyautogui into my program, which allows me to use message boxes and button options instead of being able to write anything to run, this means for yes or no options, buttons can be used instead of the user typing 'Y' or "Yes" and so on. The message boxes also look a lot more neat and professional than the message being displayed below the code, especially when markdown's surround the code. I also encountered an error when the user pressed cancel on the buttons as this would return None, which then causes string related errors, to counter this issue I said if the box == None and then gave a message saying they cancelled and moved back.